# yolox models in torchscript format

This notebook shows you how to export [yolox](https://github.com/Megvii-BaseDetection/YOLOX) models in [torchscript](https://pytorch.org/docs/stable/jit.html) format, that later on can be used as an input to [AWS SageMaker Neo](https://docs.aws.amazon.com/sagemaker/latest/dg/neo.html) compilation job or as an [AWS Panorama](https://docs.aws.amazon.com/panorama/latest/dev/index.html) model. For simplicity, we'll use the yolox-s(mall) version, but it should work also for the other model versions. 

This code is roughly based on the [`tools/export_torchscript.py`](https://github.com/Megvii-BaseDetection/YOLOX/blob/main/tools/export_torchscript.py) script. In order to be compiled by Neo or used by Panorama, the pytorch Sigmoid Linear Unit (SiLU) activation function has to be replaced by a custom implementation, as SiLU is not typically implemented in embedeed runtimes, and the compilation engine Apache TVM does not support it. This idea was taken from the [`tools/export_onnx.py`](https://github.com/Megvii-BaseDetection/YOLOX/blob/main/tools/export_onnx.py) script.

## Get the YOLOX repository and install dependencies

In [ ]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX

In [ ]:
%pip install -r ./YOLOX/requirements.txt
%pip install -v -e ./YOLOX/
%pip install cython
%pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

yolox requirements.txt does not specify the exact pytorch version, however for SageMaker Neo suppports only pytorch 1.6, 1.7, and 1.8.

In [ ]:
%pip install torch==1.8.0 torchvision==0.9.0

In [ ]:
import torch
import torchvision
print('pytorch version:', torch.__version__)
print('torchvision version:', torchvision.__version__)

## Download the pretrained yolox-s weights

YOLOX authors released model weight artifacts pretrained on the [COCO dataset](https://cocodataset.org/). The format of these pretrained models is a pytorch checkpoint file that contains the model's [state dictionary](https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict).

Refer to [YOLOX GitHub repository](https://github.com/Megvii-BaseDetection/YOLOX) for other pretrained model urls.

In [ ]:
!mkdir -p models

ckpt_url = 'https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth'
ckpt_filename = './models/yolox_s.pth'

!curl -L $ckpt_url -o $ckpt_filename

## Initialize the network

The weights contained in the state dictionary can be loaded into a neural network. We'll need a fully initialized network in order to export it in torchscript format.

In [ ]:
from torch import nn
from yolox.exp import get_exp

exp = get_exp(None, 'yolox-s')
model = exp.get_model()
ckpt = torch.load(ckpt_filename, map_location='cpu')
model.load_state_dict(ckpt['model'])

## Patch the model before exporting

As mentioned earlier, the pytorch SiLU activation function has to be replaced by a custom implementation. Also we'll disable decoding in the model head, as it is used only during training. We'll also set the model to evaluation mode (disables dropout and other training-only features).

In [ ]:
from yolox.utils import replace_module
from yolox.models.network_blocks import SiLU

model = model.eval()
model = replace_module(model, nn.SiLU, SiLU)
model.head.decode_in_inference = False

## Export model in torchscript format

Unlike a dynamic pytorch model, models saved in torchscript are static. This means that the input size of your model can not be any more dynamic, and you have to specify it at export time (now). The yolox experiment instance give you hint about the input size, in the case of yolox-s model, it is 640x640. We'll create a dummy input image of this size, as this is required when exporting the model in torchscript format. The other dimensions of the input is the batch size (1) and the channels (3 for red, green, and blue).

In [ ]:
traced_model_filename = './models/yolox_s_torchscript.pth'

input_size = [1, 3, *exp.test_size]
print('Exported model input size:', input_size)
dummy_input = torch.randn(*input_size)
traced_model = torch.jit.trace(model, dummy_input)
traced_model.save(traced_model_filename)
print('Exported model was saved to:', traced_model_filename)

## Archive the model

SageMaker Neo and Panorama both expect the model archived in a tar.gz file. 

In [ ]:
import tarfile

model_archive_filename = './models/yolox_s_torchscript.tar.gz'
with tarfile.open(model_archive_filename, "w:gz") as f:
    f.add(traced_model_filename)
print('Exported model was archived as:', model_archive_filename)

Now you can specify this archive as a Panorama model asset, or upload it to S3 and start a SageMaker Neo compilation job with it.